**Redes Neurais Convolucionais**

**Exercício #5**

Inicialmente os alunos deverão escolher entre 4 datasets:

1.   Selecionar 15 frutas distintas do dataset fruits 360 (Fácil)
2.   Selecionar todas as frutas do dataset fruits 360 (Médio). Disponível em:
https://www.dropbox.com/s/kzhuv6xim8ejxp3/fruits.zip?dl=0
3. CoronaHack -Chest X-Ray-Dataset (Médio):
Classificar apenas se existe Pneumonia ou Não, opcionalmente classificar a existência de COVID-19. Disponível em:
https://www.dropbox.com/s/zmo4c6pzwcgd9c4/562468_1022626_bundle_archive.zip?
dl=0
4. Dataset Skin Cancer MNIST: HAM10000 (difícil). Disponível em: https://www.dropbox.com/s/8o4ysiccspp46ov/skincancer-
mnist-ham10000.zip?dl=0

Com o dataset selecionado deve-se construir uma CNN para a classificação das imagens do dataset selecionado.

Instruções Gerais:

• Utilize a LetNet-5 como exemplo inicial bem como o exemplo visto em salada de aula do dataset minst;

• Inicie o trabalho com parte do dataset vá adicionando gradativamente mais tipos distintos de frutos;

• É permitido usar outras abordagens e arquiteturas;

• É permitido o uso de Autoencoders;

• É obrigatório o uso de CNN;

• É permitido o uso de técnicas de processamento de imagens;

• Não é permitido o uso de modelos pré-treinados;

• Faça uma boa distribuição dos dados de teste e treinamento e validação;

• Aplique os pré-processamentos necessários.

• Apresente a evolução do conjunto de treino e validação.

• Apresente a acurácia do conjunto de testes (opcional).

• O conjunto de teste deve variar entre 20% e 30% do conjunto total de treinamento e deve ter a mesma proporção da quantidade de amostras por classes de forma aleatório. Para o dataset fruits360 o conjunto precisa ser construído, em outras palavras é necessário fazer uma rotina que faça isso, selecionando imagens de forma aleatória e respeitando a proporcionalidade de amostras por
classes (todo tipo de fruta deve ter a mesma quantidade).

• Apresente a configuração (estrutura) da rede neural.

Dicas:

Utilize as instruções de leituras de diretórios de imagens do próprio Keras
(semelhante ao exemplo visto com o dataset de dígitos):

train_datagen =
preprocessing.image.ImageDataGenerator(validation_split=0.20)
train_generator = train_datagen.flow_from_directory('Dataset', batch_size=200, color_mode = 'grayscale',
target_size=(30, 30))